<a href="https://colab.research.google.com/github/chaelinoh/deep-learning/blob/main/keras)_%EA%B0%9C_%EA%B3%A0%EC%96%91%EC%9D%B4_%EA%B5%AC%EB%B6%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 806M/814M [00:06<00:00, 95.4MB/s]
100% 814M/814M [00:06<00:00, 123MB/s] 


In [2]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [3]:
!unzip -q train.zip -d .

In [10]:
import tensorflow as tf
import shutil

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

#개,고양이 사진 각 폴더로 복사
for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/' + i,'/content/dataset/cat/'+i)
  if 'dog' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/dog/'+i)




In [16]:
#tf.keras 사용해서 이미지 숫자화
#tf.keras.preprocessing.image_dataset_from_directory()

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),  #모든 이미지를 64*64픽셀로 변경
    batch_size = 64,     #한번에 모든 이미지를 넣지 않고 64개씩 뽑아서 돌려보기 위함
    subset='training',   #데이터의 80%
    validation_split=0.2,   
    seed=1234
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),  #모든 이미지를 64*64픽셀로 변경
    batch_size = 64,     #한번에 모든 이미지를 넣지 않고 64개씩 뽑아서 돌려보기 위함
    subset='validation',
    validation_split=0.2,   #데이터의 20%
    seed=1234
)

print(train_ds)

#데이터 전처리. 인풋 데이터 0~1 사이로 압축하기

def 전처리함수(i, 정답):
  i = tf.cast(i/255.0, tf.float32)
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)


"""
import matplotlib.pyplot as plt

for i, 정답 in train_ds.take(1):
  print(i)
  print(정답)
  plt.imshow(i[0].numpy().astype('uint8'))
  plt.show()
  """

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


"\nimport matplotlib.pyplot as plt\n\nfor i, 정답 in train_ds.take(1):\n  print(i)\n  print(정답)\n  plt.imshow(i[0].numpy().astype('uint8'))\n  plt.show()\n  "

In [ ]:
#모델만들기

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(64,64,3)), #칼라사진이라 (_,_,3)
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'), 
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2), #dropout fpdldj(overfitting 완화 가능)
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'), 
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid"),  #0~1 사이 확률값 출력
])

model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)      